In [ ]:
# This file is a part of https://github.com/pawansharmaaaa/Lip_Wise/ repository.
import gradio as gr

# Custom Modules
import infer
import file_check

bg_upscalers = list(file_check.REAL_ESRGAN_MODEL_URL.keys())

# Create interface
inputs_for_image = [
    gr.Image(type="filepath", label="Image"),
    gr.Audio(type="filepath", label="Audio"),
    gr.Number(value=0, label="Padding: Increase if getting black outlines"),
    gr.Checkbox(label = "Perform 3D_alignment?"),
    gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer"),
    gr.Slider(minimum=1, maximum=60, step=1, value=25, label="FPS"),
    gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False),
    gr.Slider(minimum=0.0, maximum=1.0, step=0.1, value=0.3, label="Weight"),
    gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False),
    gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus")
]
output_for_image = gr.Video(sources='upload', label="Output")

image_Interface = gr.Interface(fn=infer.infer_image, inputs=inputs_for_image, outputs=output_for_image)

inputs_for_video = [
    gr.Video(sources='upload',label="Video"),
    gr.Audio(type="filepath", label="Audio"),
    gr.Number(value=0, label="Padding: Increase if getting black outlines"),
    gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer"),
    gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False),
    gr.Slider(minimum=0.0, maximum=1.0, step=0.1, value=0.3, label="Weight"),
    gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False),
    gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus")
]
output_for_video = gr.Video(sources='upload', label="Output")

video_Interface = gr.Interface(fn=infer.infer_video, inputs=inputs_for_video, outputs=output_for_video)

# Run interface
ui = gr.TabbedInterface([image_Interface, video_Interface], ['Process Image', 'Process Video'],title="Lip-Wise",theme='gradio/monochrome')

ui.launch()

# New UI

In [ ]:
WEIGHTS = '''
For CodeFormer:
    \n\t0 for better quality, 1 for better identity
\nFor GFPGAN:
    \n\t1 for better quality, 0 for better identity
'''

In [2]:
%load_ext gradio

In [122]:
# This file is a part of https://github.com/pawansharmaaaa/Lip_Wise/ repository.
import gradio as gr

# Custom Modules
import infer
import file_check

bg_upscalers = list(file_check.REAL_ESRGAN_MODEL_URL.keys())

def render_dd(bg_upscale):
    
    return gr.Dropdown(
                        choices=bg_upscalers,
                        label="REALESRGAN Model",
                        value="RealESRGAN_x2plus",
                        info="Choose the model to use for upscaling the background.",
                        # Initially disabled and hidden
                        visible=bg_upscale
                    )

def render_weight(face_restorer):
    if face_restorer == "CodeFormer":
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.3,
                            label="CodeFormer Weight",
                            info="0 for better quality, 1 for better identity."
                        )
    else:
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.5,
                            label="GFPGAN Weight",
                            info="0 for better identity, 1 for better quality."
                        )
    
# Theme
theme = gr.themes.Base(
    primary_hue=gr.themes.Color(c100="#efe7ff", c200="#decefe", c300="#ccb7fd", c400="#ba9ffc", c50="#ffffff", c500="#a688fa", c600="#836bc3", c700="#61508e", c800="#41365d", c900="#241e30", c950="#25242a"),
    secondary_hue=gr.themes.Color(c100="#e2e1e4", c200="#c6c5c9", c300="#aba9b0", c400="#908d96", c50="#ffffff", c500="#76737e", c600="#5e5b64", c700="#46444b", c800="#302f33", c900="#1b1b1d", c950="#25242a"),
    neutral_hue=gr.themes.Color(c100="#e1e1e1", c200="#c4c4c4", c300="#a7a7a7", c400="#8c8c8c", c50="#ffffff", c500="#717171", c600="#5a5a5a", c700="#434343", c800="#2e2e2e", c900="#25242a", c950="#1b1b1b"),
    spacing_size="md",
    radius_size="lg",
).set(
    shadow_drop='*shadow_inset',
    shadow_drop_lg='*button_shadow_hover',
    body_background_fill="radial-gradient( circle farthest-corner at -4% -12.9%,  rgba(255,255,255,1) 0.3%, rgba(255,255,255,1) 90.2% );",
    # body_background_fill_dark="radial-gradient( circle farthest-corner at -4% -12.9%,  rgba(0,0,0,1) 0.3%, rgba(77,72,85,1) 90.2% );",
    body_background_fill_dark= "linear-gradient(315deg, #0cbaba 0%, #380036 74%);"
    # body_background_fill_dark= "linear-gradient(315deg, #7f5a83 0%, #0d324d 74%);"

    # body_background_fill_dark= "radial-gradient( circle farthest-corner at -4% -12.9%,  rgba(74,98,110,1) 0.3%, rgba(30,33,48,1) 90.2% );",
    # body_background_fill_dark= "-webkit-linear-gradient(to right, #2C5364, #203A43, #0F2027);",
    # body_background_fill_dark= "linear-gradient(to right, #2C5364, #203A43, #0F2027);"
)

head_html = f'''
<head class>
  <meta author="Pawan Sharma">
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
'''

# Create interface

with gr.Blocks(title='Lip-Wise', css=r"E:\Lip_Wise\style.css", head=head_html, theme=theme) as demo:
    with gr.Row(elem_classes=["row"]):
        gr.HTML(
            '''
            <header>
            <div class="header-left">
                <h1>Lip Wise</h1>
                <h2>Wise Enhancements for Wav2Lip</h2>
            </div>
            <div class="header-right">
                <img src="https://github.com/pawansharmaaaa/Lip_Wise/assets/56242483/5bc1b8af-879a-414b-b54a-db605a53c8f7" alt="Logo">
            </div>
            </header>
            '''
        )
    with gr.Tab(label="Process Image", elem_id="tab", elem_classes=["tabs"]):
        with gr.Row(elem_classes=["row"]):
            with gr.Column():
                gr.Markdown("# INPUTS")
                with gr.Accordion(label="Input Image and Audio", open=True, elem_classes=["inp_group", "accordion"]):
                    image_input = gr.Image(type="filepath", label="Image", container=True, elem_id = "inp_group")
                    audio_input = gr.Audio(type="filepath", label="Audio", container=True, elem_id = "inp_group")

                with gr.Accordion(label="Options", open=True, elem_classes=["opt_group", "accordion"]):
                    with gr.Group():
                        with gr.Row():
                            fps = gr.Slider(minimum=1, maximum=60, step=1, value=25, label="FPS", info="Desired Frames per second (FPS) of the output video.")
                            padding = gr.Slider(minimum=0, maximum=60, step=1, value=0, label="Padding", info="Increase if getting black outlines. The Value is in Pixels.")
                        with gr.Row():
                            alignment = gr.Checkbox(label = "Perform 3D_alignment", info = "This will improve the quality of the lip sync, but the output will be different from the original video.")
                            upscale_bg = gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False, info="This will improve the quality of the video, but will take longer to process.")
                        with gr.Row():
                            face_restorer = gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer", info="GFPGAN is faster, but CodeFormer is more accurate.", interactive=True)
                            bg_model = gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus", info="Choose the model to use for upscaling the background.", visible=False)
                        with gr.Row():
                            with gr.Column():
                                mel_step_size = gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False)
                                weight = render_weight(face_restorer)
                                upscale_bg.select(render_dd, upscale_bg, bg_model)
                                face_restorer.select(render_weight, face_restorer, weight)
                
                process = gr.Button(value="Process Image", variant="primary", elem_id="gen-button")


            with gr.Column():
                gr.Markdown("# OUTPUT")
                image_output = gr.Video(sources='upload', label="Output", elem_classes=["output"])

                process.click(infer.infer_image, [image_input, audio_input, padding, alignment, face_restorer, fps, mel_step_size, weight, upscale_bg, bg_model], [image_output])

    with gr.Tab(label="Process Video", elem_id="tab", elem_classes=["tabs"]):
        with gr.Row(elem_classes=["row"]):
            with gr.Column():
                gr.Markdown("# INPUTS")
                with gr.Accordion("Input Video and Audio", open=True, elem_classes=["inp_group", "accordion"]):
                    video_input = gr.Video(sources='upload',label="Video")
                    audio_input = gr.Audio(type="filepath", label="Audio")
                
                with gr.Accordion(label="Options", open=True, elem_classes=["opt_group", "accordion"]):
                    with gr.Group():
                        with gr.Column():
                            padding = gr.Slider(minimum=0, maximum=60, step=1, value=0, label="Padding", info="Increase if getting black outlines. The Value is in Pixels.", elem_id="slider")
                            face_restorer = gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer", info="GFPGAN is faster, but CodeFormer is more accurate.")
                            mel_step_size = gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False)
                            weight = render_weight(face_restorer)
                        with gr.Column():
                            upscale_bg = gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False, info="This will improve the quality of the video, but will take longer to process.")
                            bg_model = gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus", info="Choose the model to use for upscaling the background.", visible=False)
                            
                            upscale_bg.select(render_dd, upscale_bg, bg_model)
                            face_restorer.select(render_weight, face_restorer, weight)
                    
                process = gr.Button(value="Process Video", variant="primary", elem_id="gen-button")

            with gr.Column():
                gr.Markdown("# OUTPUT")
                video_output = gr.Video(sources='upload', label="Output", elem_classes=["output"])

                process.click(infer.infer_video, [video_input, audio_input, padding, face_restorer, mel_step_size, weight, upscale_bg, bg_model], [video_output])

    with gr.Tab(label="Guide", elem_id="tab", elem_classes=["tabs"]):
        with gr.Accordion(label="Tips For Better Results", open=True, elem_classes=["guide"]):
            gr.Markdown(
            """
            > - Optimal performance is achieved with a **clear image** featuring a person facing the camera, regardless of head angle. However, avoid **tilting in the z-direction** (3D-alignment can address this with certain considerations).
            > - Ensure the image contains only **one person** with a prominently visible face.
            > - Clear audio devoid of background noise enhances results significantly.
            > - Note that **higher image resolution** necessitates **additional processing time**.
            """,
            line_breaks=True)

        with gr.Accordion(label="Model Selection", open=True, elem_classes=["guide"]):
            gr.Markdown(
            """
            > ##### **CODEFORMER:**
            >**Recommended Weight:** `0 for better quality, 1 for better identity.`
            >>- CodeFormer employs a transformative architecture to restore facial features.
            >>- While relatively slower, it boasts **higher accuracy**.
            >>- Generally delivers superior results while **preserving skin texture**.
            >>- In cases of peculiar artifacts, especially around the nose, consider using GFPGAN.
            
            > ##### **GFPGAN:**
            >**Recommended Weight:** `0 for better identity, 1 for better quality.`
            >>- GFPGAN, a faster model, relies on a GAN-based framework for facial restoration.
            >>- Suggested for use **when CodeFormer exhibits undesirable artifacts**.
            >>- However, it often sacrifices skin texture fidelity.
            """,
            line_breaks=True)

        with gr.Accordion(label="3D Alignment", open=True, elem_classes=["guide"]):
            gr.Markdown(
            """
            > Enabling this feature **transforms** the image to ensure the person faces the camera directly. While enhancing lip sync quality, the output may diverge from the original video.
            """,
            line_breaks=True)

        with gr.Accordion(label="Background Upscaling", open=True, elem_classes=["guide"]):
            gr.Markdown(
            """
            > - Activating this feature **enhances video quality** but prolongs processing time.
            > - For most scenarios, RealESRGAN_x2plus is preferable due to its comparative speed.
            > - Optimal results are achieved when combined with CodeFormer, except in cases of nose-related artifacts.
            > - This feature effectively **eliminates video flickering**.
            """,
            line_breaks=True)

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


# Building theme

In [ ]:
gr.themes.builder()

In [ ]:
import gradio as gr
import file_check

def render_dd(bg_upscale):
    
    return gr.Dropdown(
                        choices=bg_upscalers,
                        label="REALESRGAN Model",
                        value="RealESRGAN_x2plus",
                        info="Choose the model to use for upscaling the background.",
                        # Initially disabled and hidden
                        visible=bg_upscale
                    )

def render_weight(face_restorer):
    if face_restorer == "CodeFormer":
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.3,
                            label="CodeFormer Weight"
                        )
    else:
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.5,
                            label="GFPGAN Weight"
                        )

bg_upscalers = list(file_check.REAL_ESRGAN_MODEL_URL.keys())

# Assuming you have necessary imports and definitions for bg_upscalers
with gr.Blocks() as ui:
    with gr.Tab(label="Process Video"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("# INPUTS")
                with gr.Group():
                    video_input = gr.Video(sources='upload', label="Video")
                    audio_input = gr.Audio(type="filepath", label="Audio")
                with gr.Group():
                    with gr.Row():
                        padding = gr.Slider(
                            minimum=1,
                            maximum=60,
                            step=1,
                            value=0,
                            label="Padding",
                            info="Increase if getting black outlines. The Value is in Pixels."
                        )
                        face_restorer = gr.Radio(
                            ["GFPGAN", "CodeFormer"],
                            value='CodeFormer',
                            label="Face Restorer",
                            info="GFPGAN is faster, but CodeFormer is more accurate."
                        )
                        mel_step_size = gr.Number(
                            value=16,
                            label="Mel Step Size",
                            interactive=False,
                            visible=False
                        )
                        weight = gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.3,
                            label="Restorer Weight"
                        )
                    with gr.Row():
                        upscale_bg = gr.Checkbox(
                            label="Upscale Background with REALESRGAN",
                            value=False,
                            info="This will improve the quality of the video, but will take longer to process."
                        )
                        bg_model = gr.Dropdown(
                            choices=bg_upscalers,
                            label="REALESRGAN Model",
                            value="RealESRGAN_x2plus",
                            info="Choose the model to use for upscaling the background.",
                            # Initially disabled and hidden
                            visible=False
                        )
                        upscale_bg.select(render_dd, upscale_bg, bg_model)

            with gr.Column():
                gr.Markdown("# OUTPUT")
                image_output = gr.Video(sources='upload', label="Output")

    live = True

ui.launch()

# ... rest of your Gradio interface



In [6]:
import gradio as gr

# Custom CSS
custom_css = """
html, body {
  font-family: JetBrains Mono, monospace;
  margin: 0;
  padding: 0;
}

.gradio-header {
  background-color: #ffffff;
  padding: 10px 20px;
  display: flex;
  justify-content: space-between;
  align-items: center;
}

.gradio-header h1 {
  font-size: 20px;
  margin: 0;
}

.gradio-input {
  margin-bottom: 15px;
}
"""

with gr.Blocks(css=custom_css, head=gr.HTML("""
        <h1>Calculator App</h1>
    """)) as demo:
    # Custom header

    # Inputs
    num1 = gr.Number(label="Number 1")
    num2 = gr.Number(label="Number 2")

    # Calculation
    def add(n1, n2):
        return n1 + n2

    # Output
    output = gr.Number(label="Result")

    num1.change(add, [num1, num2], output)
    num2.change(add, [num1, num2], output)

demo.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


# Colors

/* CSS HEX */
--jet: #383838ff;
--lemon-chiffon: #faf0caff;
--eerie-black: #1e1e1eff;
--red-cmyk: #ed1c24ff;
--pumpkin: #fa7921ff;

/* CSS HSL */
--jet: hsla(0, 0%, 22%, 1);
--lemon-chiffon: hsla(48, 83%, 89%, 1);
--eerie-black: hsla(0, 0%, 12%, 1);
--red-cmyk: hsla(358, 85%, 52%, 1);
--pumpkin: hsla(24, 96%, 55%, 1);

/* SCSS HEX */
$jet: #383838ff;
$lemon-chiffon: #faf0caff;
$eerie-black: #1e1e1eff;
$red-cmyk: #ed1c24ff;
$pumpkin: #fa7921ff;

/* SCSS HSL */
$jet: hsla(0, 0%, 22%, 1);
$lemon-chiffon: hsla(48, 83%, 89%, 1);
$eerie-black: hsla(0, 0%, 12%, 1);
$red-cmyk: hsla(358, 85%, 52%, 1);
$pumpkin: hsla(24, 96%, 55%, 1);

/* SCSS RGB */
$jet: rgba(56, 56, 56, 1);
$lemon-chiffon: rgba(250, 240, 202, 1);
$eerie-black: rgba(30, 30, 30, 1);
$red-cmyk: rgba(237, 28, 36, 1);
$pumpkin: rgba(250, 121, 33, 1);

/* SCSS Gradient */
$gradient-top: linear-gradient(0deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-right: linear-gradient(90deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-bottom: linear-gradient(180deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-left: linear-gradient(270deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-top-right: linear-gradient(45deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-bottom-right: linear-gradient(135deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-top-left: linear-gradient(225deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-bottom-left: linear-gradient(315deg, #383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);
$gradient-radial: radial-gradient(#383838ff, #faf0caff, #1e1e1eff, #ed1c24ff, #fa7921ff);

In [ ]:
# @title # Color Generator
import colorsys

def hex_to_rgb(hex_color):
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb_color):
    return ''.join([f'{i:02x}' for i in rgb_color])

def generate_colors(primary_950):
    primary_950_rgb = hex_to_rgb(primary_950[1:])  # Remove the '#' at the start
    primary_950_hls = colorsys.rgb_to_hls(*[x/255.0 for x in primary_950_rgb])

    colors = {}
    colors_rgb = {}
    for i in range(50, 1000, 50):
        weight = i / 1000.0
        lighter_color_hls = (primary_950_hls[0], max(0, min(1, weight)), primary_950_hls[2])
        lighter_color_rgb = [int(x*255.0) for x in colorsys.hls_to_rgb(*lighter_color_hls)]
        colors_rgb[f'primary_{i}'] = lighter_color_rgb
        colors[f'primary_{i}'] = '#' + rgb_to_hex(lighter_color_rgb)

    return colors, colors_rgb

primary_950 = '#382bf0'  # Example input
colors, colors_rgb = generate_colors(primary_950)
for name, color in colors.items():
    print(f'{name}: {color}')

for name, color in colors_rgb.items():
    print(f'{name}: {color}')

# Primary Hues

primary_50: #e8e7fd
primary_100: #d2cffb
primary_150: #bbb7f9
primary_200: #a59ff8
primary_250: #8f87f6
primary_300: #7870f4
primary_350: #6258f3
primary_400: #4c40f1
primary_450: #3528ef
primary_500: #1f10ee
primary_550: #1c0fd6
primary_600: #190dbe
primary_650: #160ba6
primary_700: #120a8e
primary_750: #0f0877
primary_800: #0c065f
primary_850: #090547
primary_900: #06032f
primary_950: #030117
primary_50: [232, 231, 253]
primary_100: [210, 207, 251]
primary_150: [187, 183, 249]
primary_200: [165, 159, 248]
primary_250: [143, 135, 246]
primary_300: [120, 112, 244]
primary_350: [98, 88, 243]
primary_400: [76, 64, 241]
primary_450: [53, 40, 239]
primary_500: [31, 16, 238]
primary_550: [28, 15, 214]
primary_600: [25, 13, 190]
primary_650: [22, 11, 166]
primary_700: [18, 10, 142]
primary_750: [15, 8, 119]
primary_800: [12, 6, 95]
primary_850: [9, 5, 71]
primary_900: [6, 3, 47]
primary_950: [3, 1, 23]

# Color Generator

In [28]:
import numpy as np
from scipy import interpolate
import matplotlib.colors as mcolors

def hex_to_rgb(hex_color):
    return np.array([int(hex_color[i:i+2], 16) for i in (0, 2, 4)])

def rgb_to_hex(rgb_color):
    return ''.join([f'{int(i):02x}' for i in rgb_color])

def predict_colors(input_colors):
    input_colors.reverse()  # Reverse the order of the input colors
    weights = np.array(range(100, 700, 100))
    colors_rgb = np.array([hex_to_rgb(color[1:]) for color in input_colors])

    colors_hsv = np.array([mcolors.rgb_to_hsv(color/255) for color in colors_rgb])

    predicted_colors = {f'primary_{i}': {'hex': rgb_to_hex(color), 'rgb': color} for i, color in zip(range(100, 700, 100), colors_rgb)}
    for i in [50, 700, 800, 900, 950]:
        f = interpolate.interp1d(weights, colors_hsv, axis=0, fill_value="extrapolate", bounds_error=False)
        predicted_color_hsv = f(i)
        predicted_color_rgb = (mcolors.hsv_to_rgb(predicted_color_hsv) * 255).astype(int)
        predicted_colors[f'primary_{i}'] = {'hex': '#' + rgb_to_hex(predicted_color_rgb), 'rgb': predicted_color_rgb}

    return predicted_colors

input_colors = ['#121212', '#282828', '#3f3f3f', '#575757', '#717171', '#8b8b8b']  # Example input
# input_colors.reverse()
predicted_colors = predict_colors(input_colors)
for name, color in predicted_colors.items():
    print(f'{name}: {color}')

primary_100: {'hex': '8b8b8b', 'rgb': array([139, 139, 139])}
primary_200: {'hex': '717171', 'rgb': array([113, 113, 113])}
primary_300: {'hex': '575757', 'rgb': array([87, 87, 87])}
primary_400: {'hex': '3f3f3f', 'rgb': array([63, 63, 63])}
primary_500: {'hex': '282828', 'rgb': array([40, 40, 40])}
primary_600: {'hex': '121212', 'rgb': array([18, 18, 18])}
primary_50: {'hex': '#979797', 'rgb': array([151, 151, 151])}
primary_700: {'hex': '#-4-4-4', 'rgb': array([-4, -4, -4])}
primary_800: {'hex': '#-1a-1a-1a', 'rgb': array([-26, -26, -26])}
primary_900: {'hex': '#-30-30-30', 'rgb': array([-48, -48, -48])}
primary_950: {'hex': '#-3b-3b-3b', 'rgb': array([-59, -59, -59])}


In [ ]:
import colorsys

def hex_to_rgb(hex_color):
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb_color):
    return ''.join([f'{i:02x}' for i in rgb_color])

def generate_colors(primary_950):
    primary_950_rgb = hex_to_rgb(primary_950[1:])  # Remove the '#' at the start
    primary_950_hls = colorsys.rgb_to_hls(*[x/255.0 for x in primary_950_rgb])

    colors = {}
    colors_rgb = {}
    for i in range(50, 1000, 50):
        weight = 1 - (i / 1000.0)  # Adjust the weight calculation
        lighter_color_hls = (primary_950_hls[0], max(0, min(1, weight)), primary_950_hls[2])
        lighter_color_rgb = [int(x*255.0) for x in colorsys.hls_to_rgb(*lighter_color_hls)]
        colors_rgb[f'secondary_{i}'] = lighter_color_rgb
        colors[f'secondary_{i}'] = '#' + rgb_to_hex(lighter_color_rgb)

    return colors, colors_rgb

primary_950 = '#121212'  # Example input
colors, colors_rgb = generate_colors(primary_950)
for name, color in colors.items():
    print(f'{name}: {color}')

for name, color in colors_rgb.items():
    print(f'{name}: {color}')

In [ ]:
from colorsys import hsv_to_rgb, rgb_to_hsv

def hex_to_rgb(hex_color):
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb_color):
    return ''.join([f'{i:02x}' for i in rgb_color])

def predict_color(colors, target_shade):
  """Predicts a color for a given shade based on existing shades."""
  # Convert colors to HSV for easier manipulation
  hues = [rgb_to_hsv(*hex_to_rgb(color))[0] for color in colors]

  # Calculate average hue
  average_hue = sum(hues) / len(hues)

  # Calculate average saturation and value based on available shades
  average_saturation = 0
  average_value = 0
  num_shades = 0
  for color in colors:
    rgb = hsv_to_rgb(*hex_to_rgb(color))  # Store the full RGB tuple
    if rgb[0] and rgb[1] and rgb[2]:  # Check if all RGB values exist
      average_saturation += rgb[1]
      average_value += rgb[2]
      num_shades += 1
  if num_shades > 0:  # Only calculate averages if shades have complete info
    average_saturation /= num_shades
    average_value /= num_shades

  # Calculate difference between target shade and existing shades
  differences = [(abs(target_shade - hue), color) for hue, color in zip(hues, colors)]
  closest_shade = min(differences, key=lambda x: x[0])[1]

  # Use closest shade as reference and adjust hue based on desired difference
  new_hue = average_hue + (target_shade - average_hue) * (hues[colors.index(closest_shade)] - average_hue)

  # Calculate new color in HSV and convert back to RGB and hex
  new_color_hsv = (new_hue, average_saturation, average_value)
  new_color_rgb = hsv_to_rgb(*new_color_hsv)
  new_color_hex = rgb_to_hex(new_color_rgb)

  # Ensure new color stays within RGB range (0-255)
  new_color_rgb = tuple(max(0, min(255, channel)) for channel in new_color_rgb)
  new_color_hex = rgb_to_hex(new_color_rgb)

  return new_color_rgb, new_color_hex

def generate_palette(input_colors):
  """Generates missing colors in a color palette."""
  shades = ["50", "100", "200", "300", "400", "500", "600", "700", "800", "900", "950"]
  palette = {shade: None for shade in shades}

  for shade in input_colors:
    palette[shade] = input_colors[shade]

  for shade in shades:
    if palette[shade] is None:
      color_rgb, color_hex = predict_color([palette[s] for s in shades if palette[s] is not None], int(shade))
      palette[shade] = (color_rgb, color_hex)

  return palette

# Example usage
input_colors = {
  "100": "#382bf0",
  "200": "#5e43f3",
  "300": "#7a5af5",
  "400": "#9171f8",
  "500": "#a688fa",
  "600": "#ba9ffb",
}

palette = generate_palette(input_colors)

for shade, (rgb, hex) in palette.items():
  print(f"Shade {shade}:")
  print(f"  RGB: {rgb}")
  print(f"  Hex: {hex}")
  print()

